### Training data
- Source: http://mattmahoney.net/dc/text8.zip
- Stored in: `data/train.txt`

### Analogies data
- Source: https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt
- Stored in: `data/analogies.txt`